In [ ]:
# installing 3DDFA_V2 library (It includes a face detector Faceboxes)
! git clone https://github.com/cleardusk/3DDFA_V2.git

Cloning into '3DDFA_V2'...
remote: Enumerating objects: 706, done.
remote: Total 706 (delta 0), reused 0 (delta 0), pack-reused 706
Receiving objects: 100% (706/706), 74.78 MiB | 41.13 MiB/s, done.
Resolving deltas: 100% (366/366), done.


In [ ]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd 3DDFA_V2
!sh ./build.sh

/content/3DDFA_V2
running build_ext
cythoning nms/cpu_nms.pyx to nms/cpu_nms.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/3DDFA_V2/FaceBoxes/utils/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'nms.cpu_nms' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/nms
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c nms/cpu_nms.c -o build/temp.linux-x86_64-3.7/nms/cpu_nms.o -Wno-cpp -Wno-unused-function
nms/cpu_nms.c: In function ‘__pyx_pf_3nms_7cpu_nms_2cp

In [ ]:
#importing librabies
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
import os.path
import glob
import os
from PIL import Image
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from keras import Sequential
import yaml
import os
import sys
import argparse
import imageio
from tqdm import tqdm
import yaml
from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.render import render
from utils.functions import cv_draw_landmark, get_suffix

In [ ]:
cfg = yaml.load(open('configs/mb1_120x120.yml'), Loader=yaml.SafeLoader)
onnx_flag = True
if onnx_flag:
    !pip install onnxruntime
    
    import os
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
    os.environ['OMP_NUM_THREADS'] = '4'
    from FaceBoxes.FaceBoxes_ONNX import FaceBoxes_ONNX
    from TDDFA_ONNX import TDDFA_ONNX

    face_boxes = FaceBoxes_ONNX()
    tddfa = TDDFA_ONNX(**cfg)
else:
    face_boxes = FaceBoxes()
    tddfa = TDDFA(gpu_mode=False, **cfg)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 25.9 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 


/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1523: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input
  key
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1523: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input output
  key


Convert /content/3DDFA_V2/FaceBoxes/weights/FaceBoxesProd.pth to /content/3DDFA_V2/FaceBoxes/weights/FaceBoxesProd.onnx done.


/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1523: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input alpha_shp
  key
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1523: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input alpha_exp
  key


Convert configs/bfm_noneck_v3.pkl to configs/bfm_noneck_v3.onnx done.
weights/mb1_120x120.onnx does not exist, try to convert the `.pth` version to `.onnx` online
Convert weights/mb1_120x120.pth to weights/mb1_120x120.onnx done.


In [ ]:
def fill(im,points):
    filler = cv2.convexHull(points)
    im=cv2.fillConvexPoly(im,filler , (0,0,0))
    return im
#cropping the face
def image_preprocess(frame):
    frame_bgr = frame[..., ::-1]
    boxes = face_boxes(frame_bgr)
    if boxes !=[]:
        box = boxes[0]
        boxes = [boxes[0]]
        fram1 = frame.copy()
        param_lst, roi_box_lst = tddfa(frame_bgr, boxes)
        ver = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=False)[0]
        param_lst, roi_box_lst = tddfa(frame_bgr, [ver], crop_policy='landmark')
        ver = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=False)[0]
        lst = [[round(ver[0][i]),round(ver[1][i])] for i in range(len(ver[0]))]
        lst.append([round(ver[0][17]), round(ver[1][17]+2*(ver[1][17]-ver[1][36]))])
        lst.append([round(ver[0][26]), round(ver[1][26]+2*(ver[1][26]-ver[1][45]))])
        lst = np.array(lst,dtype='int')
        frame = fill(frame,lst)
        final = fram1 -frame
        final  = final[...,::-1]
        final = final[round(box[1]):round(box[3]), round(box[0]):round(box[2])]
        left = []
        left.append(lst[1][1])
        left.append(lst[3][1])
        left.append(lst[3][0])
        left.append(int((lst[13][0]-lst[3][0])/3))
        right = []
        right.append(lst[15][1])
        right.append(lst[13][1])
        right.append(int((lst[13][0]-lst[3][0])/3))
        right.append(lst[13][0])
        frame = fill(frame,lst)
        final = fram1 -frame
        final  = final[...,::-1]
        final = final[round(box[1]):round(box[3]), round(box[0]):round(box[2])]
        return left, right
    else:
        return frame

In [ ]:
import os, sys
from FaceBoxes.FaceBoxes_ONNX import *

In [ ]:
face_boxes = FaceBoxes_ONNX(timer_flag=True)

In [ ]:
#preprocess the input
def load_dataset(img, m):
    if img.mode == 'RGBA':
      alpha = img.split()[3]
      bgmask = alpha.point(lambda x: 255-x)
      img = img.convert('RGB')
      img.paste((255,255,255), None, bgmask)
    img = img.resize((m, m), Image.ANTIALIAS)
    img = np.array(img)
    img = np.flipud(img)
    means = img.mean(axis=(0,1), dtype='float64')
    stds = img.std(axis=(0,1), dtype='float64')
    img = (img - means) / stds            
    return img

In [ ]:
#upload the models
import json
from tensorflow import keras

json_file = open('/content/gdrive/MyDrive/Models/model_1_sys.json', 'r')
#instead of model_1_sys we can use (model_2_sys, model_3_sys, model_1_dias, model_2_dias, model_3_dias)
loaded_model_json = json_file.read()
json_file.close()
loaded_model_1 =  keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model_1.load_weights("/content/gdrive/MyDrive/Models/model_1_sys.hdf5")
#instead of model_1_sys we can use (model_2_sys, model_3_sys, model_1_dias, model_2_dias, model_3_dias)
############################################################################

json_file = open('/content/gdrive/MyDrive/Models/model_2_sys.json', 'r')
#instead of model_1_sys we can use (model_1_sys, model_3_sys, model_1_dias, model_2_dias, model_3_dias)
loaded_model_json = json_file.read()
json_file.close()
loaded_model_2 =  keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model_2.load_weights("/content/gdrive/MyDrive/Models/model_2_sys.hdf5")
#instead of model_1_sys we can use (model_2_sys, model_3_sys, model_1_dias, model_2_dias, model_3_dias)

opt = keras.optimizers.Adam(learning_rate=0.0001)

loaded_model_1.compile(
    optimizer=opt,
    loss='mse',
    metrics = ['mae']
)

loaded_model_2.compile(
    optimizer=opt,
    loss='mse',
    metrics = ['mae']
)

In [ ]:
#make predictions
def predicting(left, right):

  pred_1= loaded_model_1.predict(x=[left, right])
  pred_2= loaded_model_2.predict(x=[left, right])
  return((pred_1 + pred_2)/2).tolist()

In [ ]:
#path to the test videos
files_seq = []
testing_folder ='/content/gdrive/MyDrive/Videos'
for root, subdirs, files in os.walk(testing_folder):
    for d in files:
      files_seq.append(str(d))

In [ ]:
#main function to get the predictions
def p(video_name):
  file_path = '/content/gdrive/MyDrive/Videos/'+str(video_name)
  cap = cv2.VideoCapture(file_path)
  currentframe = 0
  left = []
  right = []  
  bp = []
  while(True):
    success, frame = cap.read()
  
    if success:               
        boxes = face_boxes(frame)                   
        if len(boxes)==0:
          img1 = frame[int(3*frame.shape[0]/7):int(4*frame.shape[0]/7), int(1*frame.shape[1]/5):int(2*frame.shape[1]/5)]
          img1 = Image.fromarray(img1)
          img1 = load_dataset(img1, 32)
          left.append(img1)
          img2 = frame[int(3*frame.shape[0]/7):int(4*frame.shape[0]/7), int(3*frame.shape[1]/5):int(4*frame.shape[1]/5)]
          img2 = Image.fromarray(img2)
          img2 = load_dataset(img2, 32)
          right.append(img2)
          currentframe +=1

        else:
          for i in range(len(boxes)):
            box = np.array(boxes[i],dtype=int)
            orig_image = frame[box[1]:box[3], box[0]:box[2]]
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)
            orig_image =orig_image[...,::-1]
            img1 = orig_image[int(orig_image.shape[0]/2):int(3*orig_image.shape[0]/4), int(orig_image.shape[1]/8):int(orig_image.shape[1]/3)]
            img1 = Image.fromarray(img1)
            img1 = load_dataset(img1, 32)
            left.append(img1)
            img2 = orig_image[int(orig_image.shape[0]/2):int(3*orig_image.shape[0]/4), int(2*orig_image.shape[1]/3):int(7*orig_image.shape[1]/8)]
            img2 = Image.fromarray(img2)
            img2 = load_dataset(img2, 32)
            right.append(img2)
            currentframe +=1

        
        if currentframe == 25:
          left = np.array(left , dtype= np.uint8).reshape(-1,25,32,32,3)
          right = np.array(right, dtype= np.uint8).reshape(-1,25,32,32,3)
          bp = np.append(bp, predicting(left, right))
          currentframe = 0
          left = []
          right = []   
    else:
      break
    bp = np.array(bp).reshape(-1,1)
    bp_values_1 = pd.Series(bp[:,0], name='S-BP').astype(float) #instead of 'S-BP' we write 'D-BP' when predicting DBP 
    bp_values_1.to_csv('/content/gdrive/MyDrive/Output/comb_1_sys/CSVs/'+ str(video_name)[:-4]+'.csv', index = False)


In [ ]:
for i in files_seq:
  p(i)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 69ms/step
Detection: 1/1 forward_pass_time: 0.0151s misc: 0.0048s
Detection: 1/1 forward_pass_time: 0.0161s misc: 0.0047s
Detection: 1/1 forward_pass_time: 0.0188s misc: 0.0048s
Detection: 1/1 forward_pass_time: 0.0140s misc: 0.0045s
Detection: 1/1 forward_pass_time: 0.0144s misc: 0.0047s
Detection: 1/1 forward_pass_time: 0.0191s misc: 0.0048s
Detection: 1/1 forward_pass_time: 0.0147s misc: 0.0049s
Detection: 1/1 forward_pass_time: 0.0187s misc: 0.0049s
Detection: 1/1 forward_pass_time: 0.0144s misc: 0.0046s
Detection: 1/1 forward_pass_time: 0.0150s misc: 0.0048s
Detection: 1/1 forward_pass_time: 0.0209s misc: 0.0054s
Detection: 1/1 forward_pass_time: 0.0151s misc: 0.0049s
Detection: 1/1 forward_pass_time: 0.0185s misc: 0.0047s
Detection: 1/1 forward_pass_time: 0.0145s misc: 0.0048s
Detection: 1/1 forward_pass_time: 0.0142s misc: 0.0047s
Detection: 1/1 forward_pass_time: 0.0151s misc: 0.0074s
D